# Sandbox
For testing and developing new Cyber Security Assessment tools in an interactive and persistent development environment.

In [ ]:
import itertools
import json
import copy
import random
import warnings
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats.distributions as distr
import seaborn as sns
import pandapower
from pathlib import Path as p

from cyber.assets import Defence, Vulnerability, CommmonDefences, CyberDevice
from communication.graph import CommNode, CommEdge
from communication.network import Aggregator, Device, CommNetwork
from attackers.random_attacker import RandomAttacker
from cyber.analysis import Analyzer
from visualization import plot_communication_network

## Procedural Generation
### Abstract Tree
Consists of Devices and Aggregators. 
* Aggregators (internal nodes) require a **Hard** amount of effort to compromise and have a 50% chance of being compromised if the necessary effort is spent
* Devices (leaf nodes) require an **Easy** amount of effort to compromise and also have a 50% chance of being compromised if the necesssary effort is spent
* Control Center (root node) is **Very Hard** to compromise

Controllable parameters include:
* Number of devices (leaf nodes)
* Number of Entrypoints (points where cyberattacks can originate)
* Number of children per parent node (inversely proportional to redundancy)
* Random deviation in number of children
* Sibling to Sibling communication (lateral edges between nodes on the same level)

In [ ]:
seed = np.random.randint(low=0, high=52600)
np.random.seed(seed); random.seed(seed)
print(f"Seed: {np.random.get_state()[1][0]}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid=pandapower.networks.create_cigre_network_mv(with_der="all")
    print(grid)
    network = CommNetwork(n_devices=3, n_entrypoints=1, children_per_parent=0, child_no_deviation=5, 
                    network_specs=p.cwd() / "specifications" / "SmartMeter_specifications.json", 
                    # "Default_specifications.json", "SmartMeter_specifications.json", "SCADA_specifications.json", ...
                    # "WAMS_specifications.json", "Protection_specifications.json"
                    grid=grid,
                    enable_sibling_to_sibling_comm=True)
print(CommNetwork.show_tree(network.root))
print(f"Number of Components: {network.n_components}")

analyzer = Analyzer(network)

## Visualization
Plot the structure of the communication network. 

In [ ]:
# Attack Network (once) 
attacker = RandomAttacker(budget=5200, verbose=False)
network.reset()
for attempt_no in range(20):
    nodes_compromised, total_effort_spent = attacker.attack_network(network)

node_fc, node_ec, line_ec, node_to_pos = plot_communication_network(network, palette="tab10") # "tab10" (default), "Set2", "Paired", "flare"

### Electrical Grid
Visualize the electrical grid that the communication network is connected to.

In [ ]:
import inspect
import matplotlib.path as mpath
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.transforms import Affine2D
from matplotlib.collections import PatchCollection
from collections.abc import Iterable

class ElectricalPatchMaker():

    def __init__(self, symbol="trafo", **kwargs) -> None:
        methods = {method_name:method for method_name, method in inspect.getmembers(ElectricalPatchMaker, predicate=inspect.isfunction)}
        if symbol in methods:
            self.symbol = symbol
            self.patch, self.centroid = methods[self.symbol](self, **kwargs)
        else:
            raise ValueError(f"The symbol '{symbol}' is not currently supported.")
    
    def bus(self, x0=0, y0=0, size=1, **kwargs):
        circle = mpath.Path.circle(center=(x0, y0), radius=size)
        bus_patch = mpatches.PathPatch(circle, **kwargs)
        centroid = (x0, y0)
        return bus_patch, centroid

    def load(self, x0=0, y0=0, size=1, **kwargs):
        offset = np.array([x0, y0])
        A = np.array([-math.sqrt(3)/2, 0.5])*size+offset
        B = np.array([math.sqrt(3)/2, 0.5])*size+offset
        C = np.array([0.0, -1.0])*size+offset
        load_path = mpath.Path([A, B, C, A], [1,2,2,79])
        load_patch = mpatches.PathPatch(load_path, **kwargs)
        return load_patch, (x0, y0)
    
    def trafo(self, x0=0, y0=0, size=1, **kwargs):
        circle1 = mpath.Path.circle(center=(x0-size/2,y0), radius=size)
        circle2 = mpath.Path.circle(center=(x0+size/2,y0), radius=size)
        trafo_path = mpath.Path.make_compound_path(circle1, circle2)
        trafo_patch = mpatches.PathPatch(trafo_path, **kwargs)
        # Return Patch and its Centroid
        centroid = (x0,y0)
        return trafo_patch, centroid

    def ext_grid(self, x0=0, y0=0, size=1, **kwargs):
        ext_grid_patch = mpatches.Rectangle((x0-size/2, y0-size/2), width=size, height=size,
                                         hatch="xxx", **kwargs)
        # Return Patch and its Centroid
        centroid = x0, y0
        return ext_grid_patch, centroid

    def line_switch(self, x0=0, y0=0, size=1, open=True, **kwargs):
        kwargs["fc"] = "white" if open else kwargs.get("ec", "black")
        line_path = mpath.Path(vertices=[(x0-size/2, y0-size/2), (x0+size/2, y0-size/2),
                                         (x0+size/2, y0+size/2), (x0-size/2, y0+size/2),
                                         (x0-size/2, y0-size/2)], codes=[1, 2, 2, 2, 79])
        line_patch = mpatches.PathPatch(line_path, **kwargs)
        # Return Patch and its Centroid
        centroid = x0, y0
        return line_patch, centroid
    
    def sgen(self, x0=0, y0=0, size=1, joinstyle="bevel", **kwargs):
        t = size * 0.8
        circle = mpath.Path.circle(center=(x0,y0), radius=size)
        triangles = mpath.Path(vertices=[(x0-t,y0+t/2), (x0,y0+t/2), (x0-t/2, y0-t/2), (x0-t,y0+t/2), # Triangle (Down)
                                         (x0,y0-t/2), (x0+t,y0-t/2), (x0+t/2, y0+t/2), (x0,y0-t/2), # Triangle (Up)
                                         (x0-t,y0+t/2), (x0+t,y0+t/2), # Top Horizontal
                                         (x0-t,y0-t/2), (x0+t,y0-t/2),], # Bottom Horizontal
                                codes=[1,2,2,2,1,2,2,2,1,2,1,2])
        sgen_path = mpath.Path.make_compound_path(circle, triangles)
        sgen_patch = mpatches.PathPatch(sgen_path, joinstyle=joinstyle, **kwargs)
        # Return Patch and its Centroid
        centroid = (x0, y0)
        return sgen_patch, centroid
    
    @staticmethod
    def make_collection(symbol, x, y, size, **kwargs):
        patches = []
        centroids = []
        size_iterable = isinstance(size, Iterable)
        try:
            for idx in range(len(x)):
                x0, y0 = x[idx], y[idx]
                patch_size = size[idx] if size_iterable else size
                patch_maker = ElectricalPatchMaker(symbol=symbol, x0=x0, y0=y0, size=patch_size)
                patch, centroid = patch_maker.patch, patch_maker.centroid
                patches.append(patch)
                centroids.append(centroid)
        except:
            raise ValueError(f"'{symbol}' not currently supported by Patch Maker")
        # Note: Hatching does NOT work with Patch Collections
        return PatchCollection(patches, **kwargs), centroids
    
class CustomHandler(object):

    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        # centroid = orig_handle.centroid
        patch = orig_handle.patch
        patch.set_transform(handlebox.get_transform())
        handlebox.add_artist(patch)
        return patch

symbols = ["trafo", "ext_grid", "line_switch", "sgen", "load", "bus"]
nrows = len(symbols) // 2 + (1 if len(symbols) % 2 != 0 else 0)
ncols = min(2, 1 + (len(symbols) // 2))
fig, axes = plt.subplots(figsize=(ncols*5,nrows*5), sharex=True, sharey=True,
                         nrows=nrows, ncols=ncols)
for i, patch_maker in enumerate(symbols):
    pm = ElectricalPatchMaker(symbol=patch_maker, edgecolor="black", facecolor="None", lw=3)
    ax = axes[i] if nrows == 1 else axes[i // 2, i % 2]
    ax.add_patch(pm.patch)
    
    ax.plot([pm.centroid[0]], [pm.centroid[1]], marker="*", markersize=5, color="black", zorder=10)
    ax.set(xlim=(-2, 2), ylim=(-2, 2), title=patch_maker, xticks=[], yticks=[])
plt.show()


In [ ]:
def update_connected_equipment(network, kind="bus"):
    grid = network.grid
    equip_df = getattr(grid, kind)
    if kind in network.equip_to_device:
        eqp_idx_to_device = network.equip_to_device[kind]
        is_compromised = {}
        for eqp_idx, device_ids in eqp_idx_to_device.items():
            for device_id in device_ids:
                device = network.id_to_node[device_id]
                if device.is_compromised:
                    # If any connected device is compromised, mark equipment as compromised
                    is_compromised[eqp_idx] = True
                    break
        equip_df["Connected"] = equip_df.index.isin(eqp_idx_to_device)
        equip_df["Compromised"] = equip_df.index.isin(is_compromised)
    else:
        equip_df["Connected"] = False
        equip_df["Compromised"] = False
    return equip_df

for kind in ["switch", "load", "gen", "sgen", "bus", "storage", "line", "motor", "trafo"]:
    equip_df = update_connected_equipment(network, kind=kind)
    if kind == "load":
        display(equip_df)

In [ ]:
from matplotlib.transforms import CompositeGenericTransform, Affine2D, ScaledTranslation
from collections import defaultdict

def rotate_to_align(start, end):
    """
    Given 2 points on a straight line, find the rotation angle (in radians) needed to
    align a symbol (pointing up) to the line
    """
    x1, y1 = start
    x2, y2 = end
    angle = math.atan((y2 - y1)/(x2 - x1))
    return angle

def place_along_line(start, end, pos=0.5):
    """
    Given 2 points on a straight line, return a position
    some fraction (0 to 1) along it.
    """
    if pos < 0 or pos > 1.0:
        raise ValueError(f"Position {pos} is not between 0.0 and 1.0")
    x1, y1 = start
    x2, y2 = end
    return x1+(x2 - x1)*pos, y1+(y2-y1)*pos

def add_symbol(ax, grid, symbol, distance, rotation, displace=True, **kwargs):
    df = getattr(grid, symbol)
    # How many instances of this symbol this bus has (e.g. 2x load)
    counts = df.bus.value_counts()
    current_count = {}
    for i, bus_idx in enumerate(df.bus):
        patch_kwargs = {k:v[i] if isinstance(v, list) else v for k,v in kwargs.items()}
        x, y = grid.bus_geodata.loc[bus_idx, ["x", "y"]]
        current_count[bus_idx] = current_count[bus_idx] + 1 if bus_idx in current_count else 0
        count = current_count[bus_idx]
        total_count = counts[bus_idx]
        if not displace:
            patch_kwargs["alpha"] = 1.0 if count == 0 else 1/total_count
        patch_maker = ElectricalPatchMaker(symbol=symbol, x0=x, y0=y, **patch_kwargs)

        # Translate symbol some distance from its bus
        translation = Affine2D().translate(distance, 0)
        translated_pos = (patch_maker.centroid[0] + distance, patch_maker.centroid[1])

        # Rotate symbol about its center (ensures it is upright at the end)
        extra_rotation = count*np.pi/16 if displace else 0
        off_rotate = Affine2D().rotate_around(*translated_pos, -rotation-extra_rotation)

        # Rotate around bus
        rel_rotate = Affine2D().rotate_around(*patch_maker.centroid, rotation+extra_rotation)

        # Apply transformation to symbol (and update centroid)
        transform = translation + off_rotate + rel_rotate + ax.transData
        patch_maker.patch.set_transform(transform)
        patch_maker.centroid = rel_rotate.transform_point(translated_pos)
        patch, (x0, y0) = patch_maker.patch, patch_maker.centroid
        # Draw line from bus to symbol (behind everything else)
        ax.plot([x, x0], [y, y0], color="black", lw=1, zorder=-10)
        # Add patch to Axis (to draw it)
        ax.add_patch(patch)
        if not displace and total_count > 1:
            ax.annotate(text:=f"x{total_count}", xytext=(x0+(distance/2)*(len(text)-1), y0), xy=(x0,y0), zorder=-10)
    legend_entry = ElectricalPatchMaker(symbol=symbol, x0=15, y0=5, size=10, lw=2,
                                        fc="white", ec="black")
    return ax, legend_entry

def add_buses(ax, grid, s=200, **kwargs):
    bus = ElectricalPatchMaker("bus")
    ax.scatter(x=grid.bus_geodata.x, y=grid.bus_geodata.y,
               marker=bus.patch.get_path(), s=s,
               label=grid.bus_geodata.index, **kwargs)
    if s >= 200: # Marker must be big enough to show text
        for bus_idx in grid.bus_geodata.index:
            x, y = grid.bus_geodata.loc[bus_idx].x, grid.bus_geodata.loc[bus_idx].y
            ax.annotate(bus_idx, xy=(x,y), zorder=20, color="white", ha="center", va="center")
    legend_entry = ElectricalPatchMaker(symbol="bus", x0=15, y0=5, size=10, lw=2,
                                        fc="white", ec="black")
    return ax, legend_entry

def add_transformers(ax, grid, **kwargs):
    startx, starty = grid.bus_geodata.loc[grid.trafo.hv_bus].x, grid.bus_geodata.loc[grid.trafo.hv_bus].y
    endx, endy = grid.bus_geodata.loc[grid.trafo.lv_bus].x, grid.bus_geodata.loc[grid.trafo.lv_bus].y
    for i, (x0, y0, x1, y1) in enumerate(zip(startx, starty, endx, endy)):
        patch_kwargs = {k:v[i] if isinstance(v, list) else v for k,v in kwargs.items()}
        # Plot line between buses that transformer connects
        plt.plot([x0, x1], [y0, y1], color="black", zorder=-1)

        # Create Transformer Symbol (2 nested circles)
        trafo = ElectricalPatchMaker(symbol="trafo", x0=(x1+x0)/2, y0=(y1+y0)/2, **patch_kwargs)

        # Rotate the Transformer symbol to align with the line it is on
        rotate = Affine2D().rotate_around(*trafo.centroid, rotate_to_align((x0,y0), (x1,y1)))
        transform = rotate + ax.transData
        trafo.patch.set_transform(transform)

        ax.add_patch(trafo.patch)
    legend_entry = ElectricalPatchMaker(symbol="trafo", x0=12, y0=4, size=5, lw=2,
                                        fc="white", ec="black")
    return ax, legend_entry

def add_line_switches(ax, grid, **kwargs):
    line_switches = grid.switch[grid.switch.et == "l"]
    for idx in line_switches.index:
        # Switch Element (placed towards a bus, on a specific line)
        line_switch = line_switches.loc[idx]

        # Find line that this switch is placed at
        line_with_switch = grid.line.loc[line_switch.element]
        
        # Find the start and end points of the switch's line
        from_bus, to_bus = line_with_switch.from_bus, line_with_switch.to_bus
        end_point_buses = grid.bus_geodata.loc[[from_bus,to_bus]]
        x0, x1 = end_point_buses.x.values
        y0, y1 = end_point_buses.y.values
        xpos, ypos = place_along_line((x0,y0),(x1, y1), pos=0.8 if to_bus == line_switch.bus else 0.2)

        # Create Line Switch Element (Patch)
        line_switch = ElectricalPatchMaker(symbol="line_switch", x0=xpos, y0=ypos,
                                           open=not line_switch.closed, **kwargs)
        
        # Transform to align with the line
        rotate = Affine2D().rotate_around(*line_switch.centroid, rotate_to_align((x0,y0), (x1,y1)))
        transform = rotate + ax.transData
        line_switch.patch.set_transform(transform)
        line_switch.centroid = rotate.transform_point(line_switch.centroid)

        # Display the Line Switch
        ax.add_patch(line_switch.patch)
    legend_entry = ElectricalPatchMaker(symbol="line_switch", x0=12, y0=5, size=10, lw=2,
                                        fc="white", ec="black")
    return ax, legend_entry

def color_by_comm(kind, connected_color="purple", compromised_color="red"):
    trafo_df = update_connected_equipment(network, kind=kind)
    trafo_df["Color"] = "black"
    trafo_df.Color[trafo_df.Connected] = connected_color
    trafo_df.Color[trafo_df.Compromised] = compromised_color
    return list(trafo_df.Color)

def plot_physical_grid(network:CommNetwork,
                       size=0.2, distance=0.5, displace=True,
                       ext_grid_rotation=np.pi/2, gen_rotation=np.pi/2, load_rotation=-np.pi/2):
    grid = network.grid

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,10))
    
    # Buses
    ax, bus = add_buses(ax, grid, cmap="jet", c=grid.bus.vn_kv,
                        ec=color_by_comm("bus"), s=size*1000, zorder=11)

    # Lines
    startx, starty = grid.bus_geodata.loc[grid.line.from_bus].x, grid.bus_geodata.loc[grid.line.from_bus].y
    endx, endy = grid.bus_geodata.loc[grid.line.to_bus].x, grid.bus_geodata.loc[grid.line.to_bus].y
    for x0, y0, x1, y1 in zip(startx, starty, endx, endy):
        plt.plot([x0, x1], [y0, y1], color="black", zorder=-1)

    # Transformers (placed on line)
    ax, trafo = add_transformers(ax, grid, size=size*0.8,
                                 ec=color_by_comm("trafo"), fc="white", zorder=10)

    # Line Switches (placed on line)
    ax, line_switch = add_line_switches(ax, grid, size=size, ec="black", zorder=10)
    
    # Static Generators
    ax, sgen = add_symbol(ax, grid, symbol="sgen", distance=distance,
                          rotation=gen_rotation, displace=displace,
                          size=size, fc="white", ec=color_by_comm("sgen"), zorder=10)

    # Loads
    ax, load = add_symbol(ax, grid, symbol="load", distance=distance, 
                          rotation=load_rotation, displace=displace,
                          size=size, fc="white", ec=color_by_comm("load"), zorder=10)
    
    # External Grid
    ax, ext_grid = add_symbol(ax, grid, symbol="ext_grid", distance=distance,
                              rotation=ext_grid_rotation, displace=displace, lw=1,
                              size=size*2, ec=color_by_comm("ext_grid"), fc="white", zorder=10)
    ax.set(aspect="equal", xticks=[], yticks=[])
    
    # Legend (with custom symbols)
    legend_map = {"Bus":bus, "Generator":sgen, "Load":load, "Transformer":trafo,
                  "External Grid":ext_grid, "Switch":line_switch}
    labels, handles = zip(*sorted(zip(*(legend_map.keys(), legend_map.values())), key=lambda t: t[0]))
    nrows = 1 + (len(labels) // 3)
    ax.legend(labels=labels, handles=handles, loc="lower center",
               bbox_to_anchor=(0.5, -0.04*nrows), ncol=min(len(labels), 3), 
               handler_map={patch_maker:CustomHandler() for patch_maker in handles},
               title="Legend", fancybox=True, fontsize='large', title_fontsize='larger')
    
    plt.show()

plot_physical_grid(network, size=0.2, distance=0.5, displace=True)

In [ ]:
import warnings
from collections import Counter
from pandapower.plotting import draw_collections, create_bus_collection, create_line_collection, \
                                create_trafo_collection, create_line_switch_collection, \
                                create_sgen_collection, create_load_collection, create_ext_grid_collection
from matplotlib import colormaps
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D

def modal_value(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,8))
grid = network.grid
N = len(np.unique(grid.bus.vn_kv))
cmap_list = [(volt) for volt in np.unique(grid.bus.vn_kv)]
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "", category=FutureWarning)

    bus_patch = Line2D([0], [0], marker="8", markersize=10, markerfacecolor="white",
                       markeredgecolor="black", color="None")
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    bus_pc = create_bus_collection(grid, zorder=10,
        size=0.2, patch_type="poly8", 
        infofunc=lambda id:grid.bus.loc[id].name,
        # cbar_title="Bus Voltage (kV)",
        linewidth=2, color=["black"]*len(grid.bus))
        # cmap="jet", norm=Normalize(vmin=0, vmax=grid.bus.vn_kv.max()*1.1),
        # z=grid.bus.vn_kv)
    bus_coords = list(zip(grid.bus_geodata.loc[:, "x"].values, grid.bus_geodata.loc[:, "y"].values))

    ext_patch = ElectricalPatchMaker(symbol="ext_grid", x0=8, size=10, lw=2, fc="white", ec="black")
    ext_grid_pc = create_ext_grid_collection(grid, size=0.3)
    
    trafo_patch = ElectricalPatchMaker(symbol="trafo", x0=12, y0=4, size=5, lw=2, fc="white", ec="black")
    trafo_pc = create_trafo_collection(grid, zorder=9,
        size=0.2, color="black",
        infofunc=lambda id:grid.bus.loc[id].name)
    
    switch_patch = ElectricalPatchMaker(symbol="line_switch", x0=8, size=10, lw=2, fc="white", ec="black")
    line_switch_pc = create_line_switch_collection(grid, zorder=8,
        size=0.2, distance_to_bus=0.5)
    
    gen_patch = ElectricalPatchMaker(symbol="sgen", x0=12, y0=4, size=10, lw=1, fc="white", ec="black")
    gen_pc = create_sgen_collection(grid, zorder=9, linewidth=1,
        size=0.2, color="black", patch_facecolor="white", patch_edgecolor="black", orientation=0)

    load_patch= ElectricalPatchMaker(symbol="load", size=10, x0=12, lw=1, fc="white", ec="black")
    load_pc = create_load_collection(grid, zorder=8, linewidth=1,
        size=0.2, color="black", patch_facecolor="white", patch_edgecolor="black", orientation=np.pi)
    
    line_pc = create_line_collection(grid, zorder=0,
        use_bus_geodata=True, color="black")
    
    legend_map = {"Bus":bus_patch, "Generator":gen_patch, "Load":load_patch, "Transformer":trafo_patch,
                  "External Grid":ext_patch, "Switch":switch_patch}

draw_collections([bus_pc, ext_grid_pc, trafo_pc, line_switch_pc, gen_pc, load_pc, line_pc],
                 ax=ax, plot_colorbars=True, set_aspect=True, axes_visible=(True, True))

labels, handles = zip(*sorted(zip(*(legend_map.keys(), legend_map.values())), key=lambda t: t[0]))
fig.legend(labels=labels, handles=handles, loc="lower center", bbox_to_anchor=(0.5, -0.1), ncol=1 + (len(labels) // 3),
           handler_map={trafo_patch:CustomHandler(), ext_patch:CustomHandler(), switch_patch:CustomHandler(),
                        gen_patch:CustomHandler(), load_patch:CustomHandler()},
           title="Legend", fancybox=True, fontsize='large', title_fontsize='larger')

# id_map = {equip.id: v for equip, v in network.equip_to_device.items()}
# for bus_id in grid.bus_geodata.index:
#     comm_ids = id_map[f"bus_{bus_id}"]

#     # Color is determined by most common Device color attached to the equipment
#     fcs, ecs = [], []
#     for comm_id in comm_ids:
#         device = network.id_to_node[comm_id]
#         pos = node_to_pos[device]
#         fcs.append(node_fc[pos])
#         ecs.append(node_ec[pos])
#     fc, ec = modal_value(fcs), modal_value(ecs)

#     # Label equipment w. Communication Device ID (or range of IDs)
#     label = f"{comm_id}" if len(comm_ids) == 1 else f"{min(comm_ids)}-{max(comm_ids)}"
#     x, y = grid.bus_geodata.loc[bus_id, ["x", "y"]]
#     plt.annotate(label,
#                  xy=(x,y), xytext=(x-1.0, y+0.5), # Place text to top-left of Bus
#                  xycoords="data", 
#                  zorder=-1,
#                  # Draw a rounded box (colored) around the text
#                  bbox=dict(boxstyle="round,pad=0.3",
#                            fc=fc, ec=ec, lw=1),
#                  # Connect the box to the Bus with a dashed line
#                  arrowprops=dict(arrowstyle="-", linestyle="--"))

## Analysis

### Monte Carlo
Build an approximate profile of the network's cyber security by launching many cyber attacks. The higher N_ATTACKS the more precise the resulting distribution is, however this comes at the cost of increased computation time.
The more nodes are compromised, the more successful the attack.

#### Active Graph Only
Only perform Monte Carlo simulation on the currently active network.

In [ ]:
N_ATTACKS = 1000
BUDGET = 52
ATTACKER_VARIANT = RandomAttacker
compromised_array, effort_array = analyzer.monte_carlo_analysis(n_attacks=N_ATTACKS, attacker_variant=ATTACKER_VARIANT, budget=BUDGET)
analyzer.plot_monte()

#### Varied Parameter
Perform monte carlo simulation while varying particular parameter, such as the level of redundancy in the network. 

In [ ]:
import os
import multiprocess as mp
N_ATTACKS = 1000
N_DEVICES = 30
BUDGET = 52
SPEC = p.cwd() / "specifications" / "SmartMeter_specifications.json" 
SEED = np.random.randint(low=0, high=52600)
N_ENTRYPOINTS = 1 # Total budget is multiplied by this!
MIN_CHILDREN = 2
MAX_CHILDREN = N_DEVICES
CHILD_NO_STEP = 2
CHILD_NO_DEVIATION = 0
no_of_children = np.arange(MIN_CHILDREN, MAX_CHILDREN, CHILD_NO_STEP)
network_specs = dict(n_devices=N_DEVICES,
                     n_entrypoints=N_ENTRYPOINTS,
                     network_specs=SPEC,
                     child_no_deviation=CHILD_NO_DEVIATION,
                     enable_sibling_to_sibling_comm=True)

compromised_array, effort_array = analyzer.monte_carlo_multi_analysis(seed, "children_per_parent", no_of_children, budget=BUDGET, n_attacks=N_ATTACKS, **network_specs)
analyzer.plot_monte()

### Static Analysis
Given an infinite budget, breaksdown the probability of compromising components in the network. The resulting probabilities are exact (except for floating point precision issues) but do not scale well to larger communication networks (> 5 nodes). Useful as a static feature of a communication network. 

In [ ]:
n_probs = analyzer.static_analysis(show_paths=False, verbose=True)
analyzer.plot_static()

In [ ]:
# Adjacency Matrix
# Does not handle self-loops / backtracking
# Consequently, probabilities will differ from combinatorial approach

def superscript(num:int):
    sup_map = {0: f"\N{SUPERSCRIPT ZERO}", 1: f"\N{SUPERSCRIPT ONE}", 2: f"\N{SUPERSCRIPT TWO}", 3: f"\N{SUPERSCRIPT THREE}", 4: f"\N{SUPERSCRIPT FOUR}", 
           5: f"\N{SUPERSCRIPT FIVE}",  6: f"\N{SUPERSCRIPT SIX}", 7: f"\N{SUPERSCRIPT SEVEN}", 8: f"\N{SUPERSCRIPT EIGHT}", 9: f"\N{SUPERSCRIPT NINE}"}
    return "".join(sup_map[digit] for digit in map(int, str(num)))

np.set_printoptions(precision=2, floatmode="maxprec_equal")
nodes = sorted(network.graph.nodes(), key=lambda node: node.id)
prob_lookup = [node.get_prob_to_compromise() for node in nodes]
print(f"Probabilities: {prob_lookup}")


A = nx.adjacency_matrix(network.graph, nodelist=nodes, weight="p").todense()
n_probs = {}
oldA = np.eye(A.shape[0])
for i in range(len(nodes)):
    newA = oldA@A
    # np.fill_diagonal(newA, val=0)
    print(f"A{superscript(i+1)}\n", newA)
    n_probs[i+1] = np.triu(newA, k=1).sum()
    print(f"A{superscript(i+1)}: {n_probs[i+1]}")
    oldA = newA

In [ ]:
# Mutually Exclusive Approach
# Assumes you can jump and independently attack any node (i.e. ignores communication connections!)

time_required = 0.0
nodes = network.graph.nodes()
node_probs = {node: node.get_prob_to_compromise() for node in nodes}

n_probs = {}
all_nodes = set(nodes)
cumulative = 0.0
for n_devices in range(network.n_components, 0, -1):
    n_probs[n_devices] = cumulative
    for combination in itertools.combinations(nodes, n_devices):
        probability_to_compromise = 1.0
        combination = set(combination)
        missing_nodes = all_nodes.difference(combination)
        for node in combination:
            probability_to_compromise *= node_probs[node]
        for node in missing_nodes:
            probability_to_compromise *= (1 - node_probs[node])
        n_probs[n_devices] += probability_to_compromise 
    cumulative += n_probs[n_devices]
print("\n".join(f"{k} devices: {v}" for k,v in sorted(n_probs.items(), key=lambda item: item[0])))
print("Sum:", sum(n_probs.values()))

In [ ]:
# If the probability of compromising all components is the same,
# we can use the Binomial distribution function
# Takes: 12.6 µs
N = network.n_components
k = 2
p = 0.5
cumulative = 0.0
for k in range(N, 0, -1):
    prob = math.comb(N, k)*math.pow(p, k)*math.pow(1-p,N-k)
    print(f"{k} Devices: {cumulative + prob}")
    cumulative += prob

In [ ]:
import scipy.stats.distributions as distr
distr_lookup = {
    "TruncNorm": distr.truncnorm, # Continuous, loc=mean (float), scale=standard deviation (float)
    "Exponential": distr.expon, # Continuous, scale = 1 / lambda (float)
    "Gamma": distr.gamma, # Continuous, a = shape parameter (integer)
    "Bernoulli": distr.bernoulli, # Discrete
}
n_attacks = 20
is_successful = distr.bernoulli(0.5).rvs(size=n_attacks).astype(bool)
time_taken = distr.expon(scale=0.0).rvs(size=n_attacks)[is_successful]
print(f"Successful Attacks {sum(is_successful)}/{n_attacks}\nTime Taken per Successful Attack: {time_taken}")

## Communication Network Specifications
Explores how we can supply structured information to our procedural network generation algorithm. Includes information such as the types of components and defences we expect to see in the communication network.

In [ ]:
# seed = np.random.randint(low=0, high=52600)
seed = 27194
print(f"Seed: {seed}")
np.random.seed(seed)
pcn = CommNetwork(n_devices=15, n_entrypoints=1, children_per_parent=5, child_no_deviation=1,
                  network_specs="SmartMeterNetworkSpecifications.json",
                  enable_sibling_to_sibling_comm=True)


## Power System Component Association

In [ ]:
import inspect
import warnings
import numpy as np
import pandapower as pp
import pandapower.networks as grids
grid = pp.create_empty_network()
grid_filter = lambda module: inspect.isfunction(module) and not module.__name__.startswith("_")
grid_map = {grid_name:grid_creator for grid_name, grid_creator in \
            inspect.getmembers(grids, predicate=grid_filter)}
grid_options = list(grid_map.keys())
print(", ".join(grid_map.keys()))
CHOSEN_GRID = "mv_oberrhein" # "create_cigre_network_mv" # Can be None
kwargs = dict(scenario="generation", include_substations=True) #  dict(with_der="all")
with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid_name = np.random.choice(grid_options) if CHOSEN_GRID is None else CHOSEN_GRID
    print(f"Grid: {grid_name}")
    grid = grid_map[grid_name](**kwargs)
    print(grid)
    # Controllable
    n_controllable = sum(getattr(grid, attr).shape[0] for attr in ["gen", "shunt", "trafo", "switch"])
    print(f"No. of controllable elements: {n_controllable} (generators, shunts, transformers and switches)")
    # Sensor-Only
    n_sensor_only = sum(getattr(grid, attr).shape[0] for attr in ["bus", "load", "line"])
    print(f"No. of sensor-only elements: {n_sensor_only} (buses, loads and lines)")
    print(f"Total (possible) no. of devices: {n_sensor_only+n_controllable} (generators, shunts, transformers and switches, buses, loads and lines)")

# grid.switch.closed = True
pp.plotting.simple_plot(grid, respect_switches=True, plot_line_switches=True, plot_loads=True, plot_gens=True, plot_sgens=True, )


In [ ]:
seed = np.random.randint(low=0, high=52600)
np.random.seed(seed); random.seed(seed)
print(f"Seed: {np.random.get_state()[1][0]}")

with warnings.catch_warnings():
    warnings.filterwarnings(action="ignore", category=FutureWarning)
    grid = pandapower.networks.mv_oberrhein(scenario="generation") # pandapower.networks.case14()
    print(grid)
    pcn = CommNetwork(n_devices=30, n_entrypoints=1, children_per_parent=0, child_no_deviation=5, 
                    network_specs=p.cwd() / "specifications" / "SCADA_specifications.json", 
                    # "Default_specifications.json", "SmartMeter_specifications.json", "SCADA_specifications.json", "WAMS_specifications.json"
                    grid=grid,
                    enable_sibling_to_sibling_comm=True)
print(CommNetwork.show_tree(pcn.root))
print(f"Number of Components: {pcn.n_components}")